In [105]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from dotenv import load_dotenv
import csv
from selenium.webdriver.support.ui import Select
from datetime import date, timedelta

In [106]:
load_dotenv()
edge_driver_path = os.getenv('EDGE_DRIVER_PATH')
driver = webdriver.Edge(service=Service(edge_driver_path))
list_kpop_mv_url = "https://kpop.daisuki.com.br/en/mvs.html?options=101&tags=dhcmoravxpnezjsl&cols=101110000&ord=5D&fbclid=IwAR2rpByeOSp6GktqkzWLI5ipwDLLit6IE3qEhdpLemqmeBtn8C6xiLRQYo8"
detail_url = "https://kpop.daisuki.com.br/mv.php?layout=1&id="

In [110]:
driver.get(list_kpop_mv_url)

In [111]:
more_button = driver.find_element(By.XPATH, '//*[@id="morebtn"]')
driver.execute_script("arguments[0].click();", more_button)

In [112]:
list_size_cbb = driver.find_element(By.XPATH, '//*[@id="hlimit"]')
select = Select(list_size_cbb)
select.select_by_visible_text('500 items')

published_date_inp_start = driver.find_element(By.XPATH, '//*[@id="fgyear"]')
published_date_inp_end = driver.find_element(By.XPATH, '//*[@id="fgyeared"]')

start_date = date(2014, 1, 1)
end_date = date(2024, 3, 28)
list_popup_id = []
while start_date <= end_date:
    published_date_inp_start.clear()
    published_date_inp_start.send_keys(start_date.strftime('%Y-%m-%d'))

    iteration_end_date = start_date + timedelta(days=30)
    if iteration_end_date > end_date:
        iteration_end_date = end_date

    published_date_inp_end.clear()
    published_date_inp_end.send_keys(iteration_end_date.strftime('%Y-%m-%d'))

    print(f"Start date: {start_date.strftime('%Y-%m-%d')}")
    print(f"End date: {iteration_end_date.strftime('%Y-%m-%d')}")

    update_list_btn = driver.find_element(By.XPATH, '//*[@id="btngenerate"]')
    driver.execute_script("arguments[0].click();", update_list_btn)

    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'list_tiles_item')))
    list_items = driver.find_elements(By.CLASS_NAME, 'list_tiles_item')
    print(f"Number of items: {len(list_items)}")


    for item in list_items:
        id = item.get_attribute('id')
        list_popup_id.append(int(id.replace('v', '')))

    start_date += timedelta(days=30)
    
len(list_popup_id)


Start date: 2023-11-01
End date: 2023-12-01
Number of items: 479
Start date: 2023-12-01
End date: 2023-12-31
Number of items: 396
Start date: 2023-12-31
End date: 2024-01-30
Number of items: 388
Start date: 2024-01-30
End date: 2024-02-29
Number of items: 327
Start date: 2024-02-29
End date: 2024-03-29
Number of items: 351


1941

In [150]:
class VideoData:
    def __init__(self, title, release_date, youtube_id, artist_name, artist_url, organic_views, likes, artist_organic_views, artist_debut_date, branch_physical_sales, youtube_followers):
        self.title = title
        self.release_date = release_date
        self.youtube_id = youtube_id
        self.artist_name = artist_name
        self.artist_url = artist_url
        self.organic_views = organic_views
        self.likes = likes
        self.artist_organic_views = artist_organic_views
        self.artist_debut_date = artist_debut_date
        self.branch_physical_sales = branch_physical_sales
        self.youtube_followers = youtube_followers

    def __str__(self):
        return f"title: {self.title}, release_date: {self.release_date}, youtube_id: {self.youtube_id}, artist_name: {self.artist_name}, artist_url: {self.artist_url}, organic_views: {self.organic_views}, likes: {self.likes}, artist_organic_views: {self.artist_organic_views}, artist_debut_date: {self.artist_debut_date}, branch_physical_sales: {self.branch_physical_sales}, youtube_followers: {self.youtube_followers}"

class ArtistData:
    def __init__(self, artist_organic_views, artist_debut_date, branch_physical_sales, youtube_followers):
        self.artist_organic_views = artist_organic_views
        self.artist_debut_date = artist_debut_date
        self.branch_physical_sales = branch_physical_sales
        self.youtube_followers = youtube_followers

searched_artist_data = {}
videoData_list = []

start_idx = 0
end_idx = len(list_popup_id)
idx = start_idx
for id in list_popup_id[start_idx:end_idx]:
    driver.get(detail_url + str(id))
    more_button = driver.find_element(By.XPATH, '//*[@id="morebtn"]')
    driver.execute_script("expandcontent('expcontent','morebtn');")

    type = driver.find_element(By.XPATH, '//li[contains(text(), "Type: ")]/b').text
    if type != 'Main':
        continue

    artist_name = driver.find_element(By.XPATH, '//li[contains(text(), "Artist: ")]/b/a').text
    title = driver.find_element(By.TAG_NAME, 'h1').text.replace('Video Details - ', '').replace(f' - {artist_name}', '')
    release_date = driver.find_element(By.XPATH, '//li[contains(text(), "Youtube Channel: ")]/b').text
    artist_url = driver.find_element(By.XPATH, '//li[contains(text(), "Artist: ")]/b/a').get_attribute('href')
    youtube_id = driver.find_element(By.XPATH, '//li[contains(text(), "Youtube ID:	")]/b/a').text
    
    view_like_row = driver.find_elements(By.CLASS_NAME, 'dashbottom')
    organic_views = 0
    likes = 0
    for i in range(len(view_like_row) - 29, len(view_like_row)):
        organic_views += int(driver.find_element(By.XPATH, f'/html/body/div[2]/div[2]/table/tbody/tr[{i}]/td[2]').text.replace('.', ''))
        likes += int(driver.find_element(By.XPATH, f'/html/body/div[2]/div[2]/table/tbody/tr[{i}]/td[3]').text.replace('.', ''))

    if artist_url in searched_artist_data:
        artist_data = searched_artist_data[artist_url]
    else:
        driver.get(artist_url)
        artist_organic_views = driver.find_element(By.XPATH, '//*[@id="maincontentwindow"]/article/div[1]/div[2]/table/tbody/tr[2]/td[1]/b').text
        artist_debut_date = driver.find_element(By.XPATH, '//tr[@class="dashbottom"][td[contains(text(), "Debut Day")]]/td/b').text
        try:
            branch_physical_sales = driver.find_element(By.XPATH, '//tr[@class="dashbottom"][td[contains(text(), "Brand Views")]]/td/b').text
        except:
            branch_physical_sales = ''
        try:
            youtube_followers = driver.find_element(By.XPATH, '//tr[@class="dashbottom"][td[contains(text(), "Youtube Followers")]]/td/b').text
        except:
            youtube_followers = ''
        artist_data = ArtistData(artist_organic_views, artist_debut_date, branch_physical_sales, youtube_followers)
        searched_artist_data[artist_url] = artist_data

    videoData_list.append(VideoData(title, release_date, youtube_id, artist_name, artist_url, organic_views, likes, artist_data.artist_organic_views, artist_data.artist_debut_date, artist_data.branch_physical_sales, artist_data.youtube_followers))
    
    if idx % 10 == 0:
        with open('video_data.csv', 'a', newline='') as csvfile:
            fieldnames = ['title', 'release_date', 'youtube_id', 'artist_name', 'artist_url', 'organic_views', 'likes', 'artist_organic_views', 'artist_debut_date', 'branch_physical_sales', 'youtube_followers']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            if idx == 10:
                writer.writeheader()
            for video in videoData_list:
                writer.writerow({'title': video.title, 'release_date': video.release_date, 'youtube_id': video.youtube_id, 'artist_name': video.artist_name, 'artist_url': video.artist_url, 'organic_views': video.organic_views, 'likes': video.likes, 'artist_organic_views': video.artist_organic_views, 'artist_debut_date': video.artist_debut_date, 'branch_physical_sales': video.branch_physical_sales, 'youtube_followers': video.youtube_followers})

        videoData_list.clear()

    idx += 1
    print(f"idx: {idx}, title: {title}, release_date: {release_date}, youtube_id: {youtube_id}, artist_name: {artist_name}, artist_url: {artist_url}, organic_views: {organic_views}, likes: {likes}, artist_organic_views: {artist_organic_views}, artist_debut_date: {artist_debut_date}, branch_physical_sales: {branch_physical_sales}, youtube_followers: {youtube_followers}")

idx: 1, title: Drama, release_date: SMTOWN, youtube_id: D8VEhcPeSlc, artist_name: Aespa, artist_url: https://kpop.daisuki.com.br/artists/aespa.html, organic_views: 54055863, likes: 16886612, artist_organic_views: 1.5B+, artist_debut_date: 2020-11-17, branch_physical_sales: 1.997.361.804+, youtube_followers: 5.540.000+
idx: 2, title: LALALALA, release_date: JYP Entertainment, youtube_id: dBDkYofMUs4, artist_name: Stray Kids, artist_url: https://kpop.daisuki.com.br/artists/stray-kids.html, organic_views: 52433111, likes: 29170650, artist_organic_views: 5.1B+, artist_debut_date: 2018-03-26, branch_physical_sales: 5.498.024.897+, youtube_followers: 16.100.000+
idx: 3, title: BATTER UP, release_date: BABYMONSTER, youtube_id: olDWm2veCrM, artist_name: BABYMONSTER, artist_url: https://kpop.daisuki.com.br/artists/BABYMONSTER.html, organic_views: 120312338, likes: 42001087, artist_organic_views: 351.8M+, artist_debut_date: 2023-11-27, branch_physical_sales: 832.558.139+, youtube_followers: 5.30

KeyboardInterrupt: 

In [ ]:

all_option = driver.find_element(By.XPATH, '//*[@id="table_1_length"]/label/div/div/ul/li[7]')
all_option.click()


In [ ]:
class song_dto:
    def __init__(self, release_date, artist, song_name, yt_url):
        self.release_date = release_date
        self.artist = artist
        self.song_name = song_name
        self.yt_url = yt_url

list_song = []
table_rows = driver.find_elements(By.XPATH, '//*[@id="table_1"]/tbody/tr')
cnt = 0
for row in table_rows:
    release_date = row.find_element(By.CLASS_NAME, 'column-rel_date')
    artist = row.find_element(By.CLASS_NAME, 'column-artist')
    song_name = row.find_element(By.CLASS_NAME, 'column-song')
    yt_url = row.find_element(By.TAG_NAME, 'a')
    release_date = release_date.text
    artist = artist.text
    song_name = song_name.text
    yt_url = yt_url.get_attribute('href')

    list_song.append(song_dto(release_date, artist, song_name, yt_url))
    # Print the Artist and Song Name
    cnt += 1
    print(f'idx: {cnt}, Release Date: {release_date}, Artist: {artist}, Song Name: {song_name}, Youtube URL: {yt_url}')

idx: 1, Artist: tripleS, Song Name: Just Do It (NXT), Youtube URL: https://youtu.be/s3IfM-u5hVc
idx: 2, Artist: ATEEZ, Song Name: IT's You (Yeosang, San, Wooyoung), Youtube URL: https://youtu.be/CeIODBamkQ0
idx: 3, Artist: 8TURN, Song Name: RU-PUM PUM, Youtube URL: https://youtu.be/gN2SgTVZ2aU
idx: 4, Artist: Kyuhyun, Song Name: The Story Behind, Youtube URL: https://youtu.be/YRg8K6xBhmc
idx: 5, Artist: LIMELIGHT, Song Name: TWENTY TWENTY (MiU), Youtube URL: https://youtu.be/N3ccklFWAd8
idx: 6, Artist: NMIXX, Song Name: Run For Roses, Youtube URL: https://youtu.be/j6tKIdt9K9M
idx: 7, Artist: Kino, Song Name: Fashion Style: omw to see you, Youtube URL: https://youtu.be/aDVyZoGTc5A
idx: 8, Artist: B1A4, Song Name: Rewind, Youtube URL: https://youtu.be/NWXGoLrjv2A
idx: 9, Artist: SF9, Song Name: Bibora, Youtube URL: https://youtu.be/Gft8YBmNsjA
idx: 10, Artist: ITZY, Song Name: Untouchable, Youtube URL: https://youtu.be/5e3rKInegeU
idx: 11, Artist: WAKER, Song Name: Atlantis, Youtube URL:

In [ ]:
with open('songs_overview.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Release Date", "Artist", "Song Name", "Youtube URL"])
    for song in list_song:
        writer.writerow([song.release_date, song.artist, song.song_name, song.yt_url])

In [ ]:
# luot xem, luot thich, luot sub, thoi luong, luot comment
